## Importing libraries

In [33]:
# Data lib
import pandas as pd
import numpy as np

# Viz lib
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# ML lib
from fbprophet import Prophet, models
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot, plot_plotly, plot_components_plotly

## Loading data

In [15]:
# Loading the data
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/power_consumption_project/power_cons_data_clean.csv', parse_dates=['Date_time'])

In [16]:
# Checking the info of the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 16 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Global_active_power    float64       
 1   Global_reactive_power  float64       
 2   Voltage                float64       
 3   Global_intensity       float64       
 4   Sub_metering_1         float64       
 5   Sub_metering_2         float64       
 6   Sub_metering_3         float64       
 7   Date_time              datetime64[ns]
 8   Year                   int64         
 9   Month                  int64         
 10  Week_num               int64         
 11  Month_day              int64         
 12  Week_day               int64         
 13  Year_day               int64         
 14  Hour                   int64         
 15  Season                 int64         
dtypes: datetime64[ns](1), float64(7), int64(8)
memory usage: 253.3 MB


## Preprocessing the data

In [21]:
# Grouping the data on daily interval
data_daily = data.groupby(pd.Grouper(key='Date_time', freq='D')).agg({'Global_active_power':'sum', 
                                                                            'Sub_metering_1':'sum', 
                                                                            'Sub_metering_2':'sum', 
                                                                            'Sub_metering_3':'sum'}).reset_index()

In [22]:
data_daily.head()

,Date_time,Global_active_power,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16,1209.176,0.0,546.0,4926.0
1,2006-12-17,3390.460,2033.0,4187.0,13341.0
2,2006-12-18,2203.826,1063.0,2621.0,14018.0
3,2006-12-19,1666.194,839.0,7602.0,6197.0
4,2006-12-20,2225.748,0.0,2648.0,14063.0


In [23]:
# Converting all zero values to the mean of Global_active_power
data_daily_zero_mean = data_daily.copy()
data_daily_zero_mean['Global_active_power'] = data_daily_zero_mean['Global_active_power'].replace(0, np.nan).fillna(data_daily_zero_mean['Global_active_power'].mean())

# Removing all outliers from Global_active_power
data_fbp = data_daily_zero_mean[((data_daily_zero_mean['Global_active_power'] < 3000) & (data_daily_zero_mean['Global_active_power'] > 100))]

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [24]:
# Renaming the columns
data_fbp = data_fbp.rename(columns={'Date_time':'ds', 'Global_active_power':'y'})

In [25]:
# Train - Test Split -> test data will have 90 days for validation
threshold_date = pd.to_datetime('2010-09-13')
mask = data_fbp['ds'] < threshold_date

data_fbp_train = data_fbp[mask][['ds', 'y', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
data_fbp_test = data_fbp[~mask][['ds', 'y', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]

In [26]:
print(data_fbp_train.describe())
print(data_fbp_test.describe())

                 y  Sub_metering_1  Sub_metering_2  Sub_metering_3
count  1333.000000     1333.000000     1333.000000     1333.000000
mean   1524.463553     1544.985746     1795.207052     9019.888972
std     543.560695     1529.402991     2049.668807     3796.893841
min     131.732000        0.000000        0.000000        0.000000
25%    1161.016000      429.000000      413.000000     6503.000000
50%    1534.874000     1105.000000      659.000000     9086.000000
75%    1866.886000     2174.000000     2644.000000    11568.000000
max    2987.854000    11178.000000    12109.000000    23743.000000
                 y  Sub_metering_1  Sub_metering_2  Sub_metering_3
count    90.000000       90.000000       90.000000       90.000000
mean   1518.455948     1289.077778     1377.066667     8951.066667
std     446.538184     1273.685498     1759.176779     4402.392240
min     106.006000        0.000000        0.000000        0.000000
25%    1265.843000        0.000000      475.000000     6510.50

In [27]:
# Plotting the train and test dataset
fig_split = make_subplots()
fig_split.add_trace(go.Scatter(
    x=data_fbp_train['ds'],
    y=data_fbp_train['y'],
    name='Y_Train'
    ))

fig_split.add_trace(go.Scatter(
    x=data_fbp_test['ds'],
    y=data_fbp_test['y'],
    name='Y_Test'
    ))

# Setting up the names
fig_split.update_yaxes(title_text="<b>Power</b> kilowatt [kW]")
fig_split.update_xaxes(title_text="<b>Dates")

# Setting up the layout
fig_split.update_layout(
    title={
    'text': "<b>Train and Test data split</b>",
    'y':0.9,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    autosize=True, 
    legend=dict(
                orientation="h",
                yanchor="top",
                y=1.12,
                xanchor="center",
                x=0.5))
fig_split.show()

## Models building

In [28]:
# Building the Univariate model
def build_model():
    """Define forecasting model."""
    model = Prophet( 
                    daily_seasonality=True,
                    weekly_seasonality=True,
                    yearly_seasonality=True)

    model.add_seasonality(
        name='monthly',
        period = 30.5,
        fourier_order = 5
    )

    return model

model_fbp_uni = build_model()

# Fitting/trainig the model
model_fbp_uni.fit(data_fbp_train)

In [29]:
# Building the Multivariate model
def build_model_multivariate():
    """Define multivariate forecasting model."""
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=True
    )

    model.add_seasonality(
    name='monthly',
    period = 30.5,
    fourier_order = 5
    )

    model.add_regressor('Sub_metering_1')
    model.add_regressor('Sub_metering_2')
    model.add_regressor('Sub_metering_3')

    return model

model_fbp_multivar = build_model_multivariate()

# Fitting/trainig the model
model_fbp_multivar.fit(data_fbp_train)

## Univariate model forecast and Evaluation

In [124]:
# Generating predictions - Univariate model
forecast_uni = model_fbp_uni.predict(data_fbp_test)

# Spliting predictions into train and test set
mask_uni = forecast_uni['ds'] < threshold_date

forecast_train_uni = forecast_uni[mask_uni]
forecast_test_uni = forecast_uni[~mask_uni]

In [162]:
# Ploting the Univariate model
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=data_fbp_test['ds'], y=data_fbp_test['y'], name='Test data'
))

fig.add_trace(go.Scatter(
    x=forecast_uni['ds'], y=forecast_uni['yhat'], name='Forecast data'
))

# Time range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True),
        type="date"
    )
)

# Title settings
fig.update_layout(
    title={
        'text': 'Test data vs Forecast <b>Univariate FBP Model</b> - 90 days',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

# Legend settings
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="right",
    x=1
))

fig.show()

In [127]:
# Cross validation of Univariate model - to get the cutoffs days
df_cv_uni = cross_validation(model_fbp_uni, initial='730 days', period='365 days', horizon = '365 days')
df_cv_uni.head()

INFO:fbprophet:Making 1 forecasts with cutoffs between 2009-09-12 00:00:00 and 2009-09-12 00:00:00


,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,2009-09-13,1786.122534,1181.323740,2379.083082,1570.302,2009-09-12
1,2009-09-14,1547.321796,978.798447,2086.597152,1574.858,2009-09-12
2,2009-09-15,1602.568881,989.072540,2197.494860,1456.806,2009-09-12
3,2009-09-16,1560.060963,998.946729,2109.066345,1743.162,2009-09-12
4,2009-09-17,1513.731342,931.842563,2118.657523,1223.426,2009-09-12


In [128]:
# Cross validation of Univariate model with cutoffs
cutoffs = pd.to_datetime(['2009-09-12'])
df_cv_uni2 = cross_validation(model_fbp_uni, cutoffs=cutoffs, horizon='365 days')

In [154]:
# Performace metrics Univariate model
df_pm_uni = performance_metrics(df_cv_uni)
df_pm_uni.head()

,horizon,mse,rmse,mae,mape,mdape,coverage
0,36 days,87211.190625,295.315409,221.948901,0.132456,0.107667,0.972222
1,37 days,87184.254483,295.269799,221.886169,0.132972,0.107667,0.972222
2,38 days,87305.727377,295.475426,223.111076,0.133641,0.107667,0.972222
3,39 days,91563.342025,302.594352,230.666470,0.136754,0.112295,0.972222
4,40 days,90669.795570,301.114257,226.604090,0.134567,0.112295,0.972222


## Multivariate model forecast and Evaluation

In [30]:
# Generating predictions - Multivariate model
forecast_multivar = model_fbp_multivar.predict(data_fbp_test)

# Spliting predictions into train and test set
mask_multivar = forecast_multivar['ds'] < threshold_date

forecast_train_multivar = forecast_multivar[mask_multivar]
forecast_test_multivar = forecast_multivar[~mask_multivar]

In [31]:
# Ploting the Multivariate model
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=data_fbp_test['ds'], y=data_fbp_test['y'], name='Test data'
))

fig.add_trace(go.Scatter(
    x=forecast_multivar['ds'], y=forecast_multivar['yhat'], name='Forecast data'
))

# Time range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True),
        type="date"
    )
)

# Title settings
fig.update_layout(
    title={
        'text': 'Test data vs Forecast <b>Multivar FBP Model</b> - 90 days',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

# Legend settings
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="right",
    x=1
))

fig.show()

In [157]:
# Cross validation of Multivariate model - to get the cutoffs days
df_cv_multi = cross_validation(model_fbp_multivar, initial='730 days', period='365 days', horizon = '365 days')
df_cv_multi.head()

INFO:fbprophet:Making 1 forecasts with cutoffs between 2009-09-12 00:00:00 and 2009-09-12 00:00:00


,ds,yhat,yhat_lower,yhat_upper,y,cutoff
0,2009-09-13,1877.680856,1590.523619,2159.723951,1570.302,2009-09-12
1,2009-09-14,1531.767832,1254.650065,1829.119578,1574.858,2009-09-12
2,2009-09-15,1384.807604,1109.876390,1674.567143,1456.806,2009-09-12
3,2009-09-16,1879.053745,1578.499559,2178.261659,1743.162,2009-09-12
4,2009-09-17,1248.909957,944.918563,1550.449855,1223.426,2009-09-12


In [158]:
# Cross validation of Multivariate model with cutoffs
cutoffs = pd.to_datetime(['2009-09-12'])
df_cv_multi2 = cross_validation(model_fbp_multivar, cutoffs=cutoffs, horizon='365 days')

In [159]:
# Performace metrics Multivariate model
df_pm_multi = performance_metrics(df_cv_multi)
df_pm_multi.head()

,horizon,mse,rmse,mae,mape,mdape,coverage
0,36 days,30311.318803,174.101461,136.179519,0.083261,0.072843,0.888889
1,37 days,27696.515643,166.422702,128.160036,0.078203,0.070765,0.916667
2,38 days,27901.719586,167.038078,129.633816,0.078993,0.070765,0.916667
3,39 days,28048.664104,167.477354,130.476679,0.079063,0.070765,0.916667
4,40 days,27646.757486,166.273141,128.458274,0.078153,0.069509,0.916667


## Comparing the models on test data

In [101]:
# Ploting the Multivariate and Univariate models against each other on test data
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=data_fbp_test['ds'], y=data_fbp_test['y'], name='Test data'
))

fig.add_trace(go.Scatter(
    x=forecast_multivar['ds'], y=forecast_multivar['yhat'], name='Forecast multivar'
))

fig.add_trace(go.Scatter(
    x=forecast_uni['ds'], y=forecast_uni['yhat'], name='Forecast Univar'
))

# Time range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True),
        type="date"
    )
)

# Title settings
fig.update_layout(
    title={
        'text': '<b>Test</b> vs <b>Multivar</b> Forecast vs <b>Univar</b> Forecast - 90 days',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

# Legend settings
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.005,
    xanchor="right",
    x=0.8
))

fig.show()

## Exporting model

In [42]:
# Saving the Multivariate model
import pickle

path = '/content/drive/MyDrive/Colab Notebooks/power_consumption_project/'

with open(path +'FBProphet_model_pickle.sav', 'wb') as f:
     pickle.dump(forecast_multivar, f)

## Conclusion:

The Multivariate FBP Model performs signaficantly better than the Univariate FBP Model.

The additional data which is highly correlated with Global_active_power, improves the Mutlivariate Model by more than half against the Univariate model if we observe MAPE and RMSE metrics.

A summary of the models bellow.

---

<b>The params are as follows for both models:</b>
*   Seasonality - Yearly/Monthly/Weekly/Daily
*   Target - Global Active Power
*   Train/test data - 1333 / 90 days
*   Fourier_order - 5
*   The multivar model includes Sub_metering_1/2/3
---
Comparing the performance of the models for <b>30 days</b> forecast:
*   Multivar vs Univar <b>RMSE</b>: ~160 vs ~290 - kWh on Global_active_power
*   Multivar vs Univar <b>MAPE</b>: ~7% vs ~13% - kWh on Global_active_power
---
Comparing the performance of the models for <b>60 days</b> forecast:
*   Multivar vs Univar <b>RMSE</b>: 197 vs 422 - kWh on Global_active_power
*   Multivar vs Univar <b>MAPE</b>: 9% vs 17% - kWh on Global_active_power
---
Comparing the performance of the models for <b>90 days</b> forecast:
*   Multivar vs Univar <b>RMSE</b>: 230 vs 409 - kWh on Global_active_power
*   Multivar vs Univar <b>MAPE</b>: 11% vs 24% - kWh on Global_active_power
---

### <b>Further goal should be:</b>
* Use additional params in both models to lower the RMSE and MAPE

In [3]:
import pickle

In [47]:
path = '/content/drive/MyDrive/Colab Notebooks/power_consumption_project/FBProphet/FBProphet_model_pickle.sav'

pickle_in = open(path, 'rb')
forecast_fbp = pickle.load(pickle_in)

In [48]:
# Ploting the Multivariate model
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=data_fbp_test['ds'], y=data_fbp_test['y'], name='Test data'
))

fig.add_trace(go.Scatter(
    x=forecast_fbp['ds'], y=forecast_fbp['yhat'], name='Forecast data'
))

# Time range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True),
        type="date"
    )
)

# Title settings
fig.update_layout(
    title={
        'text': 'Test data vs Forecast <b>Multivar FBP Model</b> - 90 days',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

# Legend settings
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1,
    xanchor="right",
    x=1
))

fig.show()

In [46]:
model_fbp1

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,Sub_metering_1,Sub_metering_1_lower,Sub_metering_1_upper,Sub_metering_2,Sub_metering_2_lower,Sub_metering_2_upper,Sub_metering_3,Sub_metering_3_lower,Sub_metering_3_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,extra_regressors_additive,extra_regressors_additive_lower,extra_regressors_additive_upper,monthly,monthly_lower,monthly_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2010-09-13,1296.243244,1053.730108,1681.406426,1296.243244,1296.243244,-23.956142,-23.956142,-23.956142,-69.257256,-69.257256,-69.257256,-174.145829,-174.145829,-174.145829,72.553228,72.553228,72.553228,291.315816,291.315816,291.315816,-267.359227,-267.359227,-267.359227,145.059460,145.059460,145.059460,-22.125856,-22.125856,-22.125856,-74.336965,-74.336965,-74.336965,0.0,0.0,0.0,1368.796472
1,2010-09-14,1296.668927,1491.612280,2120.804230,1296.668927,1296.668927,32.304805,32.304805,32.304805,63.186239,63.186239,63.186239,94.247656,94.247656,94.247656,531.446967,531.446967,531.446967,291.315816,291.315816,291.315816,189.738701,189.738701,189.738701,99.253509,99.253509,99.253509,21.321410,21.321410,21.321410,-70.182469,-70.182469,-70.182469,0.0,0.0,0.0,1828.115894
2,2010-09-15,1297.094610,792.764397,1431.123150,1297.094610,1297.094610,-151.696966,-151.696966,-151.696966,-62.312133,-62.312133,-62.312133,-226.316697,-226.316697,-226.316697,-180.269656,-180.269656,-180.269656,291.315816,291.315816,291.315816,-440.325797,-440.325797,-440.325797,32.903043,32.903043,32.903043,1.999363,1.999363,1.999363,-66.162082,-66.162082,-66.162082,0.0,0.0,0.0,1116.824954
3,2010-09-16,1297.520293,1171.387153,1793.896999,1297.520293,1297.520293,-49.779229,-49.779229,-49.779229,-73.354879,-73.354879,-73.354879,97.263313,97.263313,97.263313,193.509583,193.509583,193.509583,291.315816,291.315816,291.315816,-25.870795,-25.870795,-25.870795,-7.452273,-7.452273,-7.452273,-2.232678,-2.232678,-2.232678,-62.250487,-62.250487,-62.250487,0.0,0.0,0.0,1491.029876
4,2010-09-17,1297.945976,1538.398762,2158.771805,1297.945976,1297.945976,40.159737,40.159737,40.159737,257.024623,257.024623,257.024623,53.310110,53.310110,53.310110,561.670301,561.670301,561.670301,291.315816,291.315816,291.315816,350.494470,350.494470,350.494470,-5.554455,-5.554455,-5.554455,-16.169192,-16.169192,-16.169192,-58.416340,-58.416340,-58.416340,0.0,0.0,0.0,1859.616276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2010-12-07,1332.426291,1106.466662,1708.478799,1330.012195,1335.007168,17.576809,17.576809,17.576809,-79.813843,-79.813843,-79.813843,-194.199949,-194.199949,-194.199949,77.335241,77.335241,77.335241,291.315816,291.315816,291.315816,-256.436983,-256.436983,-256.436983,-140.796506,-140.796506,-140.796506,21.321410,21.321410,21.321410,161.931504,161.931504,161.931504,0.0,0.0,0.0,1409.761532
86,2010-12-08,1332.851974,599.344836,1232.685274,1330.403510,1335.529391,-151.696966,-151.696966,-151.696966,-88.981406,-88.981406,-88.981406,-379.813645,-379.813645,-379.813645,-404.168480,-404.168480,-404.168480,291.315816,291.315816,291.315816,-620.492017,-620.492017,-620.492017,-241.520325,-241.520325,-241.520325,1.999363,1.999363,1.999363,164.528683,164.528683,164.528683,0.0,0.0,0.0,928.683494
87,2010-12-09,1333.277657,1252.115588,1882.559126,1330.800325,1336.062437,25.529927,25.529927,25.529927,19.779221,19.779221,19.779221,-9.717124,-9.717124,-9.717124,234.378379,234.378379,234.378379,291.315816,291.315816,291.315816,35.592024,35.592024,35.592024,-257.862717,-257.862717,-257.862717,-2.232678,-2.232678,-2.232678,167.565934,167.565934,167.565934,0.0,0.0,0.0,1567.656036
88,2010-12-10,1333.703340,1227.433498,1851.650692,1331.166300,1336.536512,-43.298911,-43.298911,-43.298911,-70.507379,-70.507379,-70.507379,

In [45]:
# Generating predictions - Multivariate model
forecast_pkl = model_fbp1.predict(data_fbp_test)

# Spliting predictions into train and test set
mask_multivar = forecast_pkl['ds'] < threshold_date

forecast_pkl_train = forecast_pkl[mask_multivar]
forecast_pkl_test = forecast_pkl[~mask_multivar]

AttributeError: ignored